In [1500]:
import numpy as np
import pandas as pd
import string
import re
import time

from numpy import dot
from numpy.linalg import norm

from imblearn.over_sampling import SMOTE
from bert_serving.client import BertClient

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

from numpy import dot
from numpy.linalg import norm

In [5]:
first_scrape = pd.read_csv('/Users/jdobrow/Code/blurbs1.csv')

In [9]:
### Data

# BookCrossing
# http://www2.informatik.uni-freiburg.de/~cziegler/BX/

df_ratings = pd.read_csv('/Data Science/BX-CSV-Dump/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding = "latin-1")
df_users = pd.read_csv('/Data Science/BX-CSV-Dump/BX-Users.csv', sep=';', encoding='latin-1')
df_books = pd.read_csv('/Data Science/BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False, encoding = "latin-1")

df_ratings.rename(columns={'User-ID': 'User', 'Book-Rating': 'Rating'}, inplace=True)
df_users.rename(columns={'User-ID': 'User'}, inplace=True)
df_books.rename(columns={'ISBN': 'ISBN', 'Book-Title': 'Title', 'Year-Of-Publication': 'Year', 'Book-Author': 'Author'}, inplace=True)
df_books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], 1, inplace=True)

# Using only explicit ratings. 0 ratings are 'implicit' in documentation of data:
df_ratings = df_ratings[df_ratings['Rating'] > 0]

# Only keep users that have more than 9 ratings
user_ids_to_keep = []
for user in df_ratings.User.unique():
    if len(df_ratings[df_ratings.User == user]) > 9:
        user_ids_to_keep.append(user)
        
df_ratings = df_ratings[df_ratings['User'].isin(user_ids_to_keep)]
df_users = df_users[df_users['User'].isin(user_ids_to_keep)]

# Inner join df_ratings and df_books
df_ratings = df_ratings[df_ratings['ISBN'].isin(df_books.ISBN.unique())]
df_books = df_books[df_books['ISBN'].isin(df_ratings.ISBN.unique())]

df_books.reset_index(inplace=True, drop=True)
df_ratings.reset_index(inplace=True, drop=True)
df_users.reset_index(inplace=True, drop=True)

# First (of 2) batches of scraped blurbs from Goodreads
first_scrape = pd.read_csv('/Users/jdobrow/Code/blurbs1.csv')


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:30

In [933]:
# We want to add our scraped blurbs to df_books, but the scraped data is messy and
# does not line up correctly

# Our titles from the scraped URLs. They almost, but not quite match the actual titles.
url_titles = []
for book in range(len(first_scrape)):
    skip = 0
    url_split = -1
    for i in range(len(first_scrape.URL[book])):
        if first_scrape.URL[book][i] == '.':
            skip += 1
            if skip == 3:
                url_split = first_scrape.URL[book][i+1:].lower()
                url_split = re.sub(r'[^\w\s]',' ',url_split).split('_')
    if url_split == -1:
        for i in range(len(first_scrape.URL[book])):
            if first_scrape.URL[book][i] == '-':
                url_split = first_scrape.URL[book][i+1:].lower().split('-')
                break
    url_titles.append(url_split)


# The actual text of the blurb
blurbs = list(df.text)

# Add empty data to line up 
for i in range(len(df_books) - len(first_scrape)):
    url_titles.append(None)
    blurbs.append(None)

# The titles from our data, tidied up a bit to assist matching
book_titles = list(df_books['Title'][:100000])
for title in range(len(book_titles)):
    book_titles[title] = book_titles[title].lower()
    book_titles[title] = re.sub(r'[^\w\s]',' ', book_titles[title]).split()
    
compare_df = pd.DataFrame()
compare_df['URLTitle'] = url_titles
compare_df['BookTitle'] = book_titles
compare_df['Blurb'] = blurbs


In [934]:
# Checks if the title and url match each other. Matches are determined by how many
# shared words there are
def check_matched_up(url_index, book_index, data):
    try:
        count = 0
        for k in data.URLTitle[url_index]:
            if k in data.BookTitle[book_index]:
                count += 1
        if (count >= max(min(len(url_split)//2, len(title_split)), 1)):
            return True
        else:
            return False
   
    except:
        return False

In [943]:
# Iterates through the data and check if titles match up, and if not corrects them by
# shifting all of the data up or down or in some cases swapping entries.

goto = len(compare_df)
beginning = 0
while beginning < goto:

    for row in range(beginning, goto):
        if (check_matched_up(row, row, compare_df) == True):
            # everything good
            None
        elif (check_matched_up(row, row + 1, compare_df) == True) and (check_matched_up(row + 1, row, compare_df) == True):
            # 2 adjacent rows need to be swapped
            url1, url2 = compare_df.URLTitle[row], compare_df.URLTitle[row + 1]
            blurb1, blurb2 = compare_df.Blurb[row], compare_df.Blurb[row + 1]
            compare_df.iloc[row, 0], compare_df.iloc[row + 1, 0] = url2, url1
            compare_df.iloc[row, 2], compare_df.iloc[row + 1, 2] = blurb2, blurb1
            break
        elif (check_matched_up(row + 1, row, compare_df) == True):
            # Insert an empty entry into book titles to match things up
            book_titles.insert(row, None)
            url_titles.append(None)
            blurbs.append(None)
            compare_df = pd.DataFrame()
            compare_df['URLTitle'] = url_titles
            compare_df['BookTitle'] = book_titles
            compare_df['Blurb'] = blurbs
            break
        elif (check_matched_up(row, row + 1, compare_df) == True):
            # Insert an empty entry into url titles to match things up
            url_titles.insert(row, None)
            blurbs.insert(row, None)
            book_titles.append(None)
            compare_df = pd.DataFrame()
            compare_df['URLTitle'] = url_titles
            compare_df['BookTitle'] = book_titles
            compare_df['Blurb'] = blurbs
            break
        else:
            book_titles[row] = None
            url_titles[row] = None
            compare_df = pd.DataFrame()
            compare_df['URLTitle'] = url_titles
            compare_df['BookTitle'] = book_titles
            compare_df['Blurb'] = blurbs
    beginning = row + 1

compare_df = compare_df.dropna()
compare_df.reset_index(inplace=True, drop=True)

55 thirds
196 thirds
331 thirds
426 thirds
433 thirds
442 thirds
538 thirds
598 thirds
732 thirds
740 thirds
741 thirds
1151 thirds
1270 thirds
1280 thirds
1444 thirds
2392 thirds
2418 thirds
2903 thirds
2905 thirds
3027 thirds
3079 thirds
3120 thirds
3154 thirds
3422 thirds
3512 thirds
3542 thirds
3802 thirds
3846 thirds
3940 thirds
3943 thirds
4173 thirds
4183 thirds
4354 thirds
4357 thirds
4360 thirds
4391 thirds
4433 thirds
4706 thirds
4797 thirds
4868 thirds
4886 thirds
4912 thirds
4913 thirds
4920 thirds
5041 thirds
5045 thirds
5104 thirds
5115 thirds
5242 thirds
5255 thirds
5268 thirds
5611 thirds
5639 thirds
5690 thirds
5766 thirds
5773 thirds
6218 thirds
6219 thirds
6222 thirds
6281 thirds
6293 thirds
6307 thirds
6313 thirds
6428 thirds
6432 thirds
6466 thirds
6510 thirds
6518 thirds
6535 thirds
6666 thirds
6798 thirds
7050 thirds
7075 thirds
7109 thirds
7113 thirds
7126 thirds
7130 thirds
7294 thirds
7400 thirds
7402 thirds
7546 thirds
7548 thirds
7699 thirds
7701 thirds
7880

42998 thirds
43004 thirds
43011 thirds
43012 thirds
43013 thirds
43014 thirds
43018 thirds
43023 thirds
43041 thirds
43043 thirds
43050 thirds
43052 thirds
43053 thirds
43088 thirds
43094 thirds
43101 thirds
43131 thirds
43133 thirds
43135 thirds
43164 thirds
43205 thirds
43207 thirds
43217 thirds
43219 thirds
43220 thirds
43229 thirds
43232 thirds
43316 thirds
43347 thirds
43353 thirds
43468 thirds
43469 thirds
43671 thirds
43795 thirds
43810 thirds
43815 thirds
43831 thirds
43934 thirds
43948 thirds
44078 thirds
44092 thirds
44093 thirds
44134 thirds
44143 thirds
44215 thirds
44220 thirds
44368 thirds
44369 thirds
44371 thirds
44390 thirds
44556 thirds
44643 thirds
44682 thirds
44684 thirds
44761 thirds
44852 thirds
44988 thirds
45351 thirds
45352 thirds
45354 thirds
45364 thirds
45366 thirds
45408 thirds
45497 thirds
45525 thirds
45997 thirds
46029 thirds
46047 thirds
46138 thirds
46158 thirds
46196 thirds
46236 thirds
46277 thirds
46280 thirds
46290 thirds
46300 thirds
46450 thirds

66893 thirds
66904 thirds
66940 thirds
66943 thirds
66949 thirds
67016 thirds
67019 thirds
67021 thirds
67024 thirds
67027 thirds
67030 thirds
67037 thirds
67039 thirds
67057 thirds
67070 thirds
67234 thirds
67237 thirds
67241 thirds
67275 thirds
67284 thirds
67306 thirds
67308 thirds
67325 thirds
67335 thirds
67443 thirds
67445 thirds
67446 thirds
67449 thirds
67454 thirds
67466 thirds
67484 thirds
67488 thirds
67491 thirds
67498 thirds
67514 thirds
67523 thirds
67570 thirds
67578 thirds
67608 thirds
67617 thirds
67646 thirds
67694 thirds
67710 thirds
67719 thirds
67744 thirds
67751 thirds
67757 thirds
67761 thirds
67766 thirds
67769 thirds
67776 thirds
67837 thirds
67838 thirds
67839 thirds
67841 thirds
67843 thirds
67848 thirds
67849 thirds
67850 thirds
67852 thirds
67853 thirds
67855 thirds
67856 thirds
67864 thirds
67868 thirds
67871 thirds
67873 thirds
67875 thirds
67882 thirds
67887 thirds
68001 thirds
68002 thirds
68004 thirds
68006 thirds
68007 thirds
68008 thirds
68019 thirds

69173 thirds
69174 thirds
69175 thirds
69176 thirds
69177 thirds
69179 thirds
69180 thirds
69181 thirds
69182 thirds
69183 thirds
69184 thirds
69185 thirds
69186 thirds
69187 thirds
69188 thirds
69189 thirds
69190 thirds
69191 thirds
69193 thirds
69195 thirds
69197 thirds
69198 thirds
69199 thirds
69200 thirds
69201 thirds
69202 thirds
69203 thirds
69204 thirds
69205 thirds
69206 thirds
69207 thirds
69209 thirds
69210 thirds
69211 thirds
69212 thirds
69213 thirds
69214 thirds
69215 thirds
69216 thirds
69217 thirds
69218 thirds
69220 thirds
69221 thirds
69222 thirds
69223 thirds
69225 thirds
69227 thirds
69228 thirds
69229 thirds
69230 thirds
69231 thirds
69234 thirds
69236 thirds
69237 thirds
69239 thirds
69240 thirds
69241 thirds
69242 thirds
69244 thirds
69245 thirds
69246 thirds
69249 thirds
69250 thirds
69252 thirds
69253 thirds
69254 thirds
69255 thirds
69257 thirds
69258 thirds
69260 thirds
69261 thirds
69263 thirds
69264 thirds
69265 thirds
69266 thirds
69268 thirds
69269 thirds

71398 thirds
71431 thirds
71439 thirds
71450 thirds
71460 thirds
71491 thirds
71499 thirds
71530 thirds
71531 thirds
71532 thirds
71535 thirds
71540 thirds
71542 thirds
71546 thirds
71547 thirds
71597 thirds
71608 thirds
71676 thirds
71678 thirds
71680 thirds
71686 thirds
71687 thirds
71689 thirds
71903 thirds
71913 thirds
71921 thirds
71932 thirds
72001 thirds
72072 thirds
72074 thirds
72100 thirds
72108 thirds
72117 thirds
72125 thirds
72135 thirds
72164 thirds
72169 thirds
72212 thirds
72221 thirds
72440 thirds
72450 thirds
72467 thirds
72475 thirds
72524 thirds
72526 thirds
72527 thirds
72529 thirds
72531 thirds
72584 thirds
72591 thirds
72600 thirds
72604 thirds
72666 thirds
72673 thirds
72675 thirds
72807 thirds
72813 thirds
72815 thirds
72903 thirds
72913 thirds
72916 thirds
72927 thirds
72932 thirds
72934 thirds
72943 thirds
72945 thirds
72951 thirds
72968 thirds
72973 thirds
72977 thirds
72995 thirds
73061 thirds
73070 thirds
73119 thirds
73127 thirds
73185 thirds
73193 thirds

81620 thirds
81621 thirds
81623 thirds
81624 thirds
81625 thirds
81626 thirds
81627 thirds
81628 thirds
81629 thirds
81631 thirds
81632 thirds
81633 thirds
81634 thirds
81636 thirds
81639 thirds
81641 thirds
81642 thirds
81644 thirds
81646 thirds
81647 thirds
81648 thirds
81650 thirds
81653 thirds
81655 thirds
81657 thirds
81659 thirds
81660 thirds
81662 thirds
81663 thirds
81664 thirds
81666 thirds
81667 thirds
81668 thirds
81669 thirds
81670 thirds
81671 thirds
81672 thirds
81675 thirds
81676 thirds
81678 thirds
81679 thirds
81680 thirds
81681 thirds
81682 thirds
81683 thirds
81685 thirds
81687 thirds
81688 thirds
81689 thirds
81690 thirds
81692 thirds
81693 thirds
81694 thirds
81695 thirds
81696 thirds
81698 thirds
81700 thirds
81701 thirds
81703 thirds
81704 thirds
81705 thirds
81707 thirds
81708 thirds
81710 thirds
81711 thirds
81713 thirds
81714 thirds
81716 thirds
81717 thirds
81718 thirds
81719 thirds
81720 thirds
81721 thirds
81722 thirds
81723 thirds
81724 thirds
81725 thirds

87763 thirds
87766 thirds
87805 thirds
87817 thirds
87884 thirds
87895 thirds
87901 thirds
87964 thirds
87972 thirds
88046 thirds
88162 thirds
88168 thirds
88171 thirds
88192 thirds
88203 thirds
88291 thirds
88296 thirds
88299 thirds
88302 thirds
88441 thirds
88451 thirds
88460 thirds
88523 thirds
88525 thirds
88554 thirds
88557 thirds
88565 thirds
88582 thirds
88585 thirds
88612 thirds
88616 thirds
88620 thirds
88664 thirds
88675 thirds
88774 thirds
88787 thirds
88790 thirds
88829 thirds
88838 thirds
88857 thirds
88886 thirds
88890 thirds
88962 thirds
88964 thirds
88966 thirds
88974 thirds
88978 thirds
88987 thirds
88988 thirds
88991 thirds
89014 thirds
89021 thirds
89030 thirds
89038 thirds
89039 thirds
89041 thirds
89046 thirds
89056 thirds
89058 thirds
89062 thirds
89067 thirds
89090 thirds
89098 thirds
89112 thirds
89179 thirds
89183 thirds
89184 thirds
89185 thirds
89186 thirds
89187 thirds
89188 thirds
89190 thirds
89194 thirds
89198 thirds
89199 thirds
89201 thirds
89202 thirds

90354 thirds
90355 thirds
90357 thirds
90358 thirds
90359 thirds
90361 thirds
90362 thirds
90363 thirds
90364 thirds
90366 thirds
90367 thirds
90368 thirds
90369 thirds
90372 thirds
90373 thirds
90375 thirds
90376 thirds
90377 thirds
90379 thirds
90380 thirds
90381 thirds
90382 thirds
90383 thirds
90386 thirds
90387 thirds
90388 thirds
90390 thirds
90392 thirds
90393 thirds
90394 thirds
90395 thirds
90397 thirds
90399 thirds
90401 thirds
90403 thirds
90405 thirds
90406 thirds
90407 thirds
90409 thirds
90410 thirds
90412 thirds
90414 thirds
90415 thirds
90417 thirds
90418 thirds
90419 thirds
90421 thirds
90424 thirds
90562 thirds
90563 thirds
90566 thirds
90579 thirds
90581 thirds
90584 thirds
90586 thirds
90639 thirds
90650 thirds
90690 thirds
90699 thirds
90712 thirds
90720 thirds
90768 thirds
90770 thirds
90773 thirds
90777 thirds
90808 thirds
90818 thirds
90831 thirds
90907 thirds
90909 thirds
90934 thirds
90944 thirds
90960 thirds
91011 thirds
91021 thirds
91048 thirds
91077 thirds

In [1003]:
# Move blurbs from compare_df to a new dataframe to work out of
active = df_books[:100000].dropna()
blurbs = []
blurb_row = 0
for book_row in range(len(df_books)):
    cleaned_title = re.sub(r'[^\w\s]',' ', df_books['Book-Title'][book_row])
    cleaned_title = cleaned_title.lower().split()
    if cleaned_title == compare_df.BookTitle[blurb_row]:
        blurbs.append(compare_df.Blurb[blurb_row])
        if blurb_row < len(compare_df):
            blurb_row += 1
    else:
        blurbs.append(None)
active['Blurb'] = blurbs

In [1284]:
# Due to the nature of the scraped data, many of the blurbs have a repeated section.
# This block finds the repeat and deletes it.
updated_blurbs = []

for row in range(len(active)):
    # Most of the very short blurbs appear to be nonsense
    if len(active.Blurb[row]) > 100:
        try:
            # A lot of random case by case errors are happening so I catch all with a
            # try, and when it fails just go with the raw blurb since it's close enough.
            go = False
            string = active.Blurb[row][200:]
            last_i = -1
            first_i = 0
            # the regular expression search doesn't like non alphanumeric characters so this
            # searches for a chunk to find a repeating section.
            while (go == False) and (first_i < 190):
                first_char = '!'
                first_i = last_i + 1
                while not first_char.isalpha():
                    first_char = active.Blurb[row][first_i]
                    first_i += 1
                last_char = 'a'
                last_i = first_i + 1
                str_len = 0
                while (last_char.isalpha() or last_char.isspace()) and (str_len < 10):
                    str_len = last_i - first_i + 1
                    last_char = active.Blurb[row][last_i]
                    last_i += 1
                if str_len == 10:
                    go = True
                if last_i > len(active.Blurb[row]) - 2:
                    go = True
            a = re.search(r'{}'.format(active.Blurb[row][(first_i-1):(last_i-1)]), string)
            
            updated_blurbs.append(active.Blurb[row][a.start() + 200 - first_i + 1:])
            
        except:
            updated_blurbs.append(active.Blurb[row])
    else:
        updated_blurbs.append(None)
        
active['Blurb'] = updated_blurbs
active = active.dropna()
active.reset_index(inplace=True, drop=True)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: Possible set difference at position 219


In [1301]:
# Get the BERT vectors for each sentence in a blurb, and then average them

bc = BertClient()

vector_list = []
track = 0
for blurb in active.Blurb:
    raw = re.split('\. |! |\? ', blurb)
    sentences = []
    for sentence in raw:
        if (len(sentence) > 0) and not sentence.isspace():
            sentences.append(sentence)
    vectors = bc.encode(sentences)
    
    mean_vector = vectors[0]
    for i in range(len(vectors) - 1):
        mean_vector = mean_vector + vectors[i + 1]
    mean_vector = mean_vector/len(vectors)
    vector_list.append(mean_vector)
    
    track += 1
    if track % 5000 == 1:
        print(track)

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701
4801
4901
5001
5101
5201
5301
5401
5501
5601
5701
5801
5901
6001
6101
6201
6301
6401
6501
6601
6701
6801
6901
7001
7101
7201
7301
7401
7501
7601
7701
7801
7901
8001
8101
8201
8301
8401
8501
8601
8701
8801
8901
9001
9101
9201
9301
9401
9501
9601
9701
9801
9901
10001
10101
10201
10301
10401
10501
10601
10701
10801
10901
11001
11101
11201
11301
11401
11501
11601
11701
11801
11901
12001
12101
12201
12301
12401
12501
12601
12701
12801
12901
13001
13101
13201
13301
13401
13501
13601
13701
13801
13901
14001
14101
14201
14301
14401
14501
14601
14701
14801
14901
15001
15101
15201
15301
15401
15501
15601
15701
15801
15901
16001
16101
16201
16301
16401
16501
16601
16701
16801
16901
17001
17101
17201
17301
17401
17501
17601
17701
17801
17901
18001
18101
18201
18301
18401
18

In [1316]:
vectors = pd.DataFrame(vector_list)
active = pd.concat([active, vectors], axis=1)

In [1468]:
# Save the dataframe thus far to keep our progress
# active.to_csv('/Data Science/Final Capstone Files/books_with_blurbs_and_BERT.csv')

In [1359]:
# Testing how useful our BERT vector representations of the blurbs scraped from
# GoodReads are at predicting features of our data.
# Can it predict if a book was written after 1996 or not? 60% accuracy.

X = active.iloc[:,6:]
Y = active.iloc[:]['Year'] > 1996
rfc = RandomForestClassifier(n_estimators = 35)
cross_val_score(rfc, X, Y, cv=3)


array([0.61320019, 0.61082446, 0.59809927])

In [2261]:
# Accuracy at differentiating between books before 1985 and after 2002? 72%
X = active[(active.Year < 1985) | (active.Year > 2002)].iloc[:,6:]
Y = (active[(active.Year < 1985) | (active.Year > 2002)].iloc[:]['Year']) > 2000
rfc = RandomForestClassifier(n_estimators = 35)
cross_val_score(rfc, X, Y, cv=3)

array([0.7362299 , 0.72117687, 0.70773443])

In [1380]:
# Most frequent publisher in our data is Harlequin. Can we identify
# Harlequin novels.
max_len = 0
max_pub = ''
for i in active.Publisher.unique():
    x = len(active[active.Publisher == i])
    if x > max_len:
        max_len = x
        max_pub = i
print(max_len, max_pub)

1851 Harlequin


In [1415]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X, Y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
rfc = RandomForestClassifier(n_estimators = 20)
rfc.fit(X_train, y_train)
preds = rfc.predict(X_test)
confusion_matrix(preds, y_test)

In [1417]:
# 98.5% Accuracy at predicting Harlequin novels.
(15419+15694)/(15419 + 140 + 333 + 15694)

0.9850250110808586

In [2263]:
def euclidean_distance(vect1, vect2):
    return np.linalg.norm(active.iloc[vect1, 6:]-active.iloc[vect2, 6:])

def cosine_similarity(vect1, vect2):
    return dot(vect1, vect2)/(norm(vect1)*norm(vect2))

def get_user_df(user_id):
    user_ratings = pd.DataFrame()
    user_ratings['Index'] = active[active.ISBN.isin(list(df_ratings.ISBN[df_ratings.User == user_id]))].index
    user_ratings['Rating'] = list(df_ratings.Rating[df_ratings.User == user_id])
    return user_ratings

def recommend_books_by_blurb(user_df):
    x = cluster_indices(user_df.Index)
    xx = get_top_index_lists(x)

    to_check = len(active)
    for i in range(3):
        rating_list = []
        for j in xx[i]:
            rating_list.append(user_df.Rating[user_df.Index == j].iloc[0])
        xxx = euclidean_average(xx[i], rating_list)
        similarities = {}
        reverse = {}
        for i in range(to_check):
            sim = cosine_similarity(xxx, active.iloc[i, 6:])
            similarities[i] = sim
            reverse[sim] = i

        sorted_similarities = sorted(similarities.values())

        print(active['Title'][reverse[sorted_similarities[-1]]])

def cluster_indices(index_list):
    kmeans = KMeans(n_clusters = 5)
    clusters = kmeans.fit_predict(active.iloc[index_list, 6:])
    kmeans_df = pd.DataFrame()
    kmeans_df['Index'] = index_list
    kmeans_df['Cluster'] = clusters
    return kmeans_df

def euclidean_average(index_list, rating_list):
    average_vector = np.zeros(768)
    for i in range(len(index_list)):
        average_vector += active.iloc[index_list[i], 6:] * (rating_list[i] - 5.5)
    average_vector /= len(index_list)
    return average_vector

def get_top_index_lists(df):
    lengths = {}
    for i in range(5):
        lengths[i] = len(df[df.Cluster == i])
    tops = []
    list_list = []
    for k in range(3):
        for j in range(5):
            if (lengths[j] == sorted(lengths.values())[-k - 1]) and (j not in tops):
                list_list.append(list(df.Index[df.Cluster == j]))
                tops.append(j)
    return list_list
    

In [ ]:
start = time.time()
recommend_books_by_blurb(get_user_df(df_ratings.User.unique()[0]))
print('Runtime:', time.time() - start)

In [2265]:
trial = get_user_df(df_ratings.User.unique()[3])
active.Title[trial.Index]


168                                    Angels &amp; Demons
1619                  A Cold Heart: An Alex Delaware Novel
3352                 Fire Ice: A Novel from the NUMA Files
3363                                          Perish Twice
4629                                      The Devil's Code
5227     The Coffin Dancer (Lincoln Rhyme Novels (Paper...
13339           Nighttime Is My Time (Clark, Mary Higgins)
16467    The Tin Collectors : A Novel (A Shane Scully N...
16971                                       A Little Magic
21299                                         Bad Business
21322                                     Balance of Power
22466                                        City of Bones
25519                                          Mortal Prey
30977                                    Gunman's Rhapsody
33530                                   Perchance to Dream
42342                                 Tilting at Windmills
44198                            Hope of Earth (Geodysse